# Práctica 5: Sistema de Recomendación (Modelos)

---

* **Autor:** Diego Garda Porto
* **Asignatura:** Gestión de la Información
* **Grado:** Ingeniería Informática

---

## Objetivo

Implementar un filtro colaborativo basado en factorización matricial de bajo rango (se puede utilizar el algoritmo SVD de Surprise). Además, realizarán actividades complementarias para profundizar en la interpretación, evaluación y aplicación de los modelos de recomendación.


## Instrucciones
1. Implementación básica del filtro colaborativo. Utilizar el dataset de ratings
de MovieLens de 100k ratings (recommended for education and development, small)
2. Añadir uno o varios usuarios que representen a los miembros del equipo de prácticas (con ratings a un subconjunto de las películas del dataset), ajustar el filtro y mostrar las 10 mejores recomendaciones que proporciona a cada usuario añadido.
3. Analizar los sesgos estimados en el modelo ¿Qué usuarios tienden a puntuar más bajo o más alto? ¿Qué películas tienden a tener más ratings?
4. Análisis en el espacio latente a partir de los vectores de características latentes de usuarios y películas. Calcular las 10 películas más próximas a una dada, por ejemplo, “Toy Story”
4. Utilizar alguna técnica de clustering para obtener grupos de usuarios similares que pueden ser interpretados como segmentos de clientes



### 1- Instalación de numpy

In [1]:
!pip install "numpy<2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 40.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 re

### 2- Importación de paquetes

Antes de hacer esto hay que reiniciar la sesión tras la instalación anterior de numpy

In [1]:
# (Ejecutar después de reiniciar la sesión)

# 1. Instalar scikit-surprise
!pip install scikit-surprise

# 2. Importaciones de Python
import pandas as pd
import numpy as np
import os
from collections import defaultdict
import warnings

# 3. Importaciones de Surprise
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# 4. Importaciones de Scikit-learn y SciPy (para Tareas 4 y 5)
from sklearn.cluster import KMeans
from scipy.spatial.distance import cosine # Para similitud

# 5. Importaciones de Visualización (Opcional pero recomendado)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Configuración
warnings.filterwarnings('ignore')
print("Librerías instaladas e importadas correctamente.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2555157 sha256=e59c1fa7b83a354824913c12b4eac7ca3c94db95a9e5ba9603ec9d8c55a3f7ee
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise
Librerías instaladas e importadas correctamente.


### 3- Carga de datos (ml-100k) y construcción del Trainset

He utilizado el Dataset de internet MovieLens 100k que tiene 100.000 datos porque con el otro estaba teniendo problemas al cargarlo con la RAM del programa al ser un archivo más grande.

In [2]:
# 1. Cargar el Dataset (MovieLens 100k)
try:
    datos = Dataset.load_builtin('ml-100k')
    print("Dataset MovieLens 100k cargado")
except Exception as e:
    print(f"ERROR - No se pudo descargar el dataset: {e}")
    raise

# 2. Construir el Trainset completo (para el modelo base)
trainset_base = datos.build_full_trainset()
print(f"\nDatos del Trainset Base:")
print(f"Número de usuarios: {trainset_base.n_users}")
print(f"Número de películas: {trainset_base.n_items}")

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
Dataset MovieLens 100k cargado

Datos del Trainset Base:
Número de usuarios: 943
Número de películas: 1682


### 4- Recoger datos de las películas

In [3]:
# 1. Función para obtener los títulos de las películas y guardarlas
def get_movie_titles():
    try:
        home_dir = os.path.expanduser('~')
        ruta = os.path.join(
            home_dir,
            '.surprise_data',
            'ml-100k',
            'ml-100k',
            'u.item'
        )

        tirulos_peliculas = {}
        # Abrimos el archivo con la codificación correcta
        with open(ruta, 'r', encoding='ISO-8859-1') as f:
            for line in f:
                parts = line.split('|')
                id_pelicula = parts[0]
                titulo = parts[1]
                tirulos_peliculas[id_pelicula] = titulo

        return tirulos_peliculas

    except FileNotFoundError:
        print(f"ERROR: No se encontró 'u.item' en {ruta}")
        print("Asegúrate de que 'Dataset.load_builtin' se ejecutó correctamente.")
        return None
    except Exception as e:
        print(f"Error en get_movie_titles: {e}")
        return None

# 2. Crear el diccionario de títulos
tirulos_peliculas = get_movie_titles()

# 3. Mostrar ejemplos
if tirulos_peliculas:
    print("Diccionario de títulos creado con éxito.")
    print(f"Ejemplo -> ID 1: {tirulos_peliculas.get('1')}")
    print(f"Ejemplo -> ID 50: {tirulos_peliculas.get('50')}")

Diccionario de títulos creado con éxito.
Ejemplo -> ID 1: Toy Story (1995)
Ejemplo -> ID 50: Star Wars (1977)


### 5- Entrenar el modelo SVD con los datos

In [4]:
# Inicializar SVD
algoritmo_svd = SVD(n_factors=50,
                    n_epochs=20,
                    random_state=42)

# Entrenar el modelo con el trainset base
algoritmo_svd.fit(trainset_base)

print("Modelo SVD base entrenado con éxito.")

Modelo SVD base entrenado con éxito.


### 6- Añadir valoraciones de nuevos usuarios

In [5]:
# Convertir los datos de surprise a un DataFrame de Pandas
ratings_df = pd.DataFrame(datos.raw_ratings, columns=['uid', 'iid', 'rating', 'timestamp'])

# Creación de nuevos ratings para 2 nuevos usuarios
nuevos_ratings = [
    # Ratings Usuario 1 = diego
    ('diego', '50', 5.0, 0),  # 50: Star Wars
    ('diego', '181', 5.0, 0), # 181: Return of the Jedi
    ('diego', '1', 1.0, 0),   # 1: Toy Story (no le gustó)
    ('diego', '210', 4.0, 0), # 210: Indiana Jones: En busca del arca perdida
    ('diego', '269', 5.0, 0), # 269: Men in Black

    # Ratings Usuario 2
    ('manuel', '1', 5.0, 0),   # 1: Toy Story
    ('manuel', '7', 4.0, 0),   # 7: Twelve Monkeys
    ('manuel', '127', 5.0, 0), # 127: Godfather, The
    ('manuel', '56', 4.0, 0),  # 56: Pulp Fiction
    ('manuel', '95', 5.0, 0),  # 95: Aladdin
]

# Lista de los nuevos usuarios
nuevos_usuarios = ['diego', 'manuel']

# Convertir a DataFrame y añadir al original
nuevos_ratings_df = pd.DataFrame(nuevos_ratings, columns=['uid', 'iid', 'rating', 'timestamp'])
ratings_completos_df = pd.concat([ratings_df, nuevos_ratings_df], ignore_index=True)

print(f"Ratings originales: {len(ratings_df)}")
print(f"Ratings nuevos añadidos: {len(nuevos_ratings_df)}")
print(f"Total ratings (con equipo): {len(ratings_completos_df)}")

# Mostrar los ratings que hemos añadido
print("\nRatings añadidos por el equipo:")
print(nuevos_ratings_df)

Ratings originales: 100000
Ratings nuevos añadidos: 10
Total ratings (con equipo): 100010

Ratings añadidos por el equipo:
      uid  iid  rating  timestamp
0   diego   50     5.0          0
1   diego  181     5.0          0
2   diego    1     1.0          0
3   diego  210     4.0          0
4   diego  269     5.0          0
5  manuel    1     5.0          0
6  manuel    7     4.0          0
7  manuel  127     5.0          0
8  manuel   56     4.0          0
9  manuel   95     5.0          0


### 7- Reentrenar al modelo con los nuevos ratings añadidos

In [6]:
# Cargar el nuevo dataset
reader = Reader(rating_scale=(1, 5))
datos_nuevo = Dataset.load_from_df(ratings_completos_df[['uid', 'iid', 'rating']], reader)

# Construir trainset nuevo con los nuevos usuarios
trainset_nuevo = datos_nuevo.build_full_trainset()
print("\nNuevo Trainset con usuarios añadidos:")
print(f"Usuarios (nuevo): {trainset_nuevo.n_users}")
print(f"Películas (nuevo): {trainset_nuevo.n_items}")

# Reentrenar el modelo
print("\nReentrenando modelo con nuevos usuarios")
algoritmo_svd_nuevo = SVD(n_factors=50, n_epochs=20, random_state=42)
algoritmo_svd_nuevo.fit(trainset_nuevo)
print("Modelo re-entrenado con éxito.")


Nuevo Trainset con usuarios añadidos:
Usuarios (nuevo): 945
Películas (nuevo): 1682

Reentrenando modelo con nuevos usuarios
Modelo re-entrenado con éxito.


### 8- Generar recomendaciones para los nuevos usuarios añadidos

In [7]:
# Función que genera recomendaciones
def generar_recomendaciones(algoritmo, iduser, n=10):

    # Obtener el id del usuario
    try:
        id_usuario = algoritmo.trainset.to_inner_uid(user)
    except ValueError:
        print(f"Error: El usuario {id_usuario} no se encontró en el trainset.")
        return []

    # Obtener los id de las películas valoradas por el usario
    peliculas_valoradas = set(
        iid for (iid, rating) in algoritmo.trainset.ur[id_usuario]
    )

    # Predecir el rating para las películas que no ha puntuado
    predicciones = []
    for item_inner_id in algoritmo.trainset.all_items():
        if item_inner_id not in peliculas_valoradas:
            # Convertir ID interno de película a ID bruto (string)
            item_raw_id = algoritmo.trainset.to_raw_iid(item_inner_id)
            # Predecir rating
            pred = algoritmo.predict(uid=iduser, iid=item_raw_id)
            predicciones.append((item_raw_id, pred.est))

    # 4. Ordenar por rating estimado (descendente)
    predicciones.sort(key=lambda x: x[1], reverse=True)

    # 5. Devolver las Top-N (traducidas a títulos)
    top_n_with_titles = []
    for (raw_id, estimated_rating) in predicciones[:n]:
        # Usamos nuestro diccionario 'movie_titles' para traducir
        title = tirulos_peliculas.get(raw_id, "Título Desconocido")
        top_n_with_titles.append((title, estimated_rating))

    return top_n_with_titles

# --- Aquí es donde se "muestran" las recomendaciones ---
print("\n--- RECOMENDACIONES TOP-10 PARA EL EQUIPO ---")

# (La variable 'usuarios_equipo' la definimos en la Celda 12)
for user in nuevos_usuarios:
    print(f"\n--- Recomendaciones para {user} ---")
    recomendaciones = generar_recomendaciones(algoritmo_svd_nuevo, user, n=10)

    if recomendaciones:
        for i, (title, rating) in enumerate(recomendaciones):
            # Imprime: " 1. Titulo de Pelicula (Rating est: 4.52)"
            print(f"{i+1:2}. {title} (Rating est: {rating:.2f})")
    else:
        print("No se pudieron generar recomendaciones.")


--- RECOMENDACIONES TOP-10 PARA EL EQUIPO ---

--- Recomendaciones para diego ---
 1. Close Shave, A (1995) (Rating est: 4.65)
 2. Rear Window (1954) (Rating est: 4.64)
 3. Godfather, The (1972) (Rating est: 4.63)
 4. Shawshank Redemption, The (1994) (Rating est: 4.59)
 5. One Flew Over the Cuckoo's Nest (1975) (Rating est: 4.54)
 6. Raiders of the Lost Ark (1981) (Rating est: 4.50)
 7. Usual Suspects, The (1995) (Rating est: 4.50)
 8. Empire Strikes Back, The (1980) (Rating est: 4.48)
 9. Casablanca (1942) (Rating est: 4.45)
10. North by Northwest (1959) (Rating est: 4.44)

--- Recomendaciones para manuel ---
 1. Close Shave, A (1995) (Rating est: 4.92)
 2. Raiders of the Lost Ark (1981) (Rating est: 4.87)
 3. North by Northwest (1959) (Rating est: 4.86)
 4. Wrong Trousers, The (1993) (Rating est: 4.85)
 5. Star Wars (1977) (Rating est: 4.81)
 6. Rear Window (1954) (Rating est: 4.78)
 7. Shawshank Redemption, The (1994) (Rating est: 4.78)
 8. 12 Angry Men (1957) (Rating est: 4.75)
 9

### 9- Análisis de sesgos (Usuarios y Películas)


*   Usuarios que tienden a puntuar más alto y mas bajo
*   Películas que tienden a tener más ratingsd



In [8]:
print("ANÁLISIS DE SESGOS")

# Extraer biases sesgos del modelo (usuarios y películas)
sUsuarios = algoritmo_svd_nuevo.bu
sPeliculas = algoritmo_svd_nuevo.bi

print("\nSESGOS DE LOS USUARIOS")
# ANALIZAR SESGOS DE USUARIOS
lista_sUsuarios = []
for inner_id in range(trainset_nuevo.n_users):
    # Extraer el id del usuario
    raw_id = trainset_nuevo.to_raw_uid(inner_id)
    # Extraer el sesgo del usuario
    bias = sUsuarios[inner_id]
    # Guardar la información en la lista
    lista_sUsuarios.append((raw_id, bias))

sUsuarios_df = pd.DataFrame(lista_sUsuarios, columns=['Usuario', 'Sesgo (bias)'])
sUsuarios_df = sUsuarios_df.sort_values(by='Sesgo (bias)')

# Mostrar resultados para los usuarios
print("\n1. Usuarios que tienden a puntuar más bajo:")
print(sUsuarios_df.head(5).to_string(index=False))
print("\n2. Usuarios que tienden a puntuar más alto:")
print(sUsuarios_df.tail(5).sort_values(by='Sesgo (bias)', ascending=False).to_string(index=False))

print("\nSESGOS DE LAS PELÍCULAS")
# ANALIZAR SESGOS DE PELÍCULAS
lista_sPeliculas = []
for inner_id in range(trainset_nuevo.n_items):
    raw_id = trainset_nuevo.to_raw_iid(inner_id)
    # Usamos nuestro diccionario: tirulos_peliculas
    title = tirulos_peliculas.get(raw_id, "N/A")
    bias = sPeliculas[inner_id]
    lista_sPeliculas.append((raw_id, title, bias))

sPeliculas_df = pd.DataFrame(lista_sPeliculas, columns=['ID', 'Película', 'Sesgo (bias)'])
sPeliculas_df = sPeliculas_df.sort_values(by='Sesgo (bias)')

# Mostrar los resultados para las películas
print("\n3. Películas que tienden a recibir puntuaciones más bajas:")
print(sPeliculas_df[['Película', 'Sesgo (bias)']].head(5).to_string(index=False))
print("\n4. Películas que tienden a recibir puntuaciones más altas:")
print(sPeliculas_df[['Película', 'Sesgo (bias)']].tail(5).sort_values(by='Sesgo (bias)', ascending=False).to_string(index=False))

ANÁLISIS DE SESGOS

SESGOS DE LOS USUARIOS

1. Usuarios que tienden a puntuar más bajo:
Usuario  Sesgo (bias)
    181     -1.765430
    405     -1.657426
    445     -1.596894
    774     -1.422383
    206     -1.317713

2. Usuarios que tienden a puntuar más alto:
Usuario  Sesgo (bias)
    688      1.324319
    507      1.170552
    427      1.076162
    849      1.000384
    907      0.965699

SESGOS DE LAS PELÍCULAS

3. Películas que tienden a recibir puntuaciones más bajas:
                                  Película  Sesgo (bias)
Children of the Corn: The Gathering (1996)     -1.519936
                           Bio-Dome (1996)     -1.410702
        Mortal Kombat: Annihilation (1997)     -1.404321
 Lawnmower Man 2: Beyond Cyberspace (1996)     -1.399988
           Free Willy 3: The Rescue (1997)     -1.380572

4. Películas que tienden a recibir puntuaciones más altas:
                        Película  Sesgo (bias)
           Close Shave, A (1995)      1.083916
      Wrong Trousers, 

### 10- Análisis del Espacio Latente

Calculo de las 10 películas más cercanas a una dada.

In [9]:
print("TOP 10 PELÍCULAS MÁS SIMILARES A PULP FICTION (1994)")

# Película sobre la que trabajar
pelicula_objetivo = "Pulp Fiction (1994)"

# Encontrar ID
id_bruto_objetivo = None
# Recorremos diccionario de titulos para encontrar el ID
for raw_id_temp, title in tirulos_peliculas.items():
    if title == pelicula_objetivo:
        id_bruto_objetivo = raw_id_temp
        break

if id_bruto_objetivo is None:
    print(f"Error: No se pudo encontrar '{pelicula_objetivo}' en el diccionario.")
else:
    print(f"Película objetivo encontrada: {pelicula_objetivo} con ID: {id_bruto_objetivo})")

    # 3. Obtener el ID interno (inner_id)
    id_interno_objetivo = trainset_nuevo.to_inner_iid(id_bruto_objetivo)

    # 4. Obtener el vector latente (de la matriz 'qi' del modelo)
    vector_peliculas = algoritmo_svd_nuevo.qi
    vector_objetivo = vector_peliculas[id_interno_objetivo]

    # 5. Calcular similitud con todas las demás películas
    similitudes = []
    for inner_id_actual in range(trainset_nuevo.n_items):

        # Saltamos si se ha encontrado la misma película (Pulp Fiction)
        if inner_id_actual == id_interno_objetivo:
            continue

        vector = vector_peliculas[inner_id_actual]

        # Calcular similitud: 1 - distancia coseno.
        similitud = 1 - cosine(vector_objetivo, vector)

        # Traducir ID interno actual a ID bruto para el resultado
        raw_id_actual = trainset_nuevo.to_raw_iid(inner_id_actual)
        titulo = tirulos_peliculas.get(raw_id_actual, "N/A")
        similitudes.append((titulo, similitud))

    # 6. Ordenar por similitud (descendente)
    similitudes.sort(key=lambda x: x[1], reverse=True)

    # 7. Mostrar Top 10
    print(f"\n10 películas más similares a '{pelicula_objetivo}':")
    for i, (titulo, sim) in enumerate(similitudes[:10]):
        print(f"{i+1:2}. {titulo} (Similitud: {sim:.3f})")

TOP 10 PELÍCULAS MÁS SIMILARES A PULP FICTION (1994)
Película objetivo encontrada: Pulp Fiction (1994) con ID: 56)

10 películas más similares a 'Pulp Fiction (1994)':
 1. Cable Guy, The (1996) (Similitud: 0.529)
 2. GoodFellas (1990) (Similitud: 0.528)
 3. Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963) (Similitud: 0.513)
 4. Casino (1995) (Similitud: 0.469)
 5. Mighty Aphrodite (1995) (Similitud: 0.438)
 6. Chungking Express (1994) (Similitud: 0.433)
 7. Kingpin (1996) (Similitud: 0.426)
 8. Henry V (1989) (Similitud: 0.414)
 9. Smilla's Sense of Snow (1997) (Similitud: 0.405)
10. Talking About Sex (1994) (Similitud: 0.405)


### 11- Clustering de Usuarios

Obtención de grupos de usuarios similares basándonos en la proximidad geométrica de los perfiles según sus características.

In [10]:
print("CLUSTERING DE USUARIOS")

# Obtener la matriz de factores latentes de usuario (pu)
vector_usuarios = algoritmo_svd_nuevo.pu

# Número de clusters (K)
n_clusters = 5

# Aplicar K-Means
print(f"Agrupando {trainset_nuevo.n_users} usuarios en {n_clusters} clusters")
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
kmeans.fit(vector_usuarios)

# Asignar cada usuario a un cluster
asignaciones_cluster = kmeans.labels_

# Mapear usuarios (raw_id) a sus clusters
user_cluster_map = {}
for inner_id in range(trainset_nuevo.n_users):
    raw_id = trainset_nuevo.to_raw_uid(inner_id)
    cluster = asignaciones_cluster[inner_id]
    user_cluster_map[raw_id] = cluster

# Convertir a DataFrame para análisis
cluster_df = pd.DataFrame(user_cluster_map.items(), columns=['Usuario', 'Cluster'])

# RESULTADOS
print("RESULTADOS DEL CLUSTERING")
# Tamaño de cada cluster
print("1. Tamaño de cada cluster:")
print(cluster_df['Cluster'].value_counts().sort_index().to_string())

# Cluster de los usuarios agregados
print("\n2. Cluster de los usuarios agregados:")
for user in nuevos_usuarios:
    try:
        cluster = cluster_df[cluster_df['Usuario'] == user]['Cluster'].values[0]
        print(f"El usuario '{user}' pertenece al Cluster {cluster}")
    except IndexError:
        print(f"Error: No se encontró el cluster para el usuario '{user}'")

CLUSTERING DE USUARIOS
Agrupando 945 usuarios en 5 clusters
RESULTADOS DEL CLUSTERING
1. Tamaño de cada cluster:
Cluster
0    100
1    233
2    224
3    236
4    152

2. Cluster de los usuarios agregados:
El usuario 'diego' pertenece al Cluster 2
El usuario 'manuel' pertenece al Cluster 3
